# Importing Libraries

In [ ]:
import numpy as np
import pandas as pd 
import string
import warnings
warnings.filterwarnings('ignore')
import re

from textblob import TextBlob

import nltk
#nltk.download('punkt')
#nltk.download('wordnet')

import spacy


from nltk.corpus import stopwords

#Stemming
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

#Lemmetization
from nltk.stem import WordNetLemmatizer
lemmetizer = WordNetLemmatizer()

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
#Words that we do not require after tokenization in the search term and the title column as well
unwanted_words = ['Email','Sample','Letter','Template','email','sample','letter','template','to','To']
#Punctuations
punctuations = '''()-[]{};:'"\,<>./?@#$%^&*_`~'''
#Stopwords
stop_words = stopwords.words('english')

# Importing dataset and Preprocessing

In [ ]:
#Reading the data set and preprocessing
data = pd.read_excel('templates.xls')
data = data.dropna()
data = data.reset_index(drop=True)
#Replacing 'Letter' with 'Email' || Replacing '-' with ' ' in the cat column
for i in range (0,len(data)):
    data.iloc[i,0] = data.iloc[i,0].replace('Letter','Email')
    data.iloc[i,0] = data.iloc[i,0].replace('letter','Email')
    data.iloc[i,1] = data.iloc[i,1].replace('Letter','Email')
    data.iloc[i,1] = data.iloc[i,1].replace('letter','Email')
    data.iloc[i,0] = data.iloc[i,0].replace('-',' ')
data['Search_column'] = ''
data = data[["Category","Search_column", "Title", "Body"]]

#Removing the 'Download Related Samples...' from footer
for i in range (0,len(data)):
    txt = data.iloc[i,3]
    txt0 = txt.split('\nDownload ')[0]
    data.iloc[i,3] = txt0

#Removing address fields
for i in range (0,len(data)):
    list0 = data.Body[i].split('\n')
    list1 = []
    for string in list0:
        if len(string)> 50:
            list1.append(string)
    body = 'To: [Email id]\n\nSubject: [Subject]\n\n[Salutation]\n\n'
    for j in list1:
        body = body + j + '\n\n'
    body = body + '[Closure]\n\n'+'[Name/Signature]\n'+'[Designation]\n'+'[Organization]\n'
    body = body.replace('Letter','Email')
    body = body.replace('letter','email')
    #print(body)
    data.iloc[i,3] = body

#preprocessing the title column
for i in range (0,len(data)):
    tex = str(data.Title[i])
    tex_p = ''
    for char in tex:
        if (char not in punctuations):
            tex_p = tex_p+char
    #tex_p = re.sub(r'[^\w\s]','',tex)
    tex_p = tex_p.strip()
    tex_a = nltk.word_tokenize(tex_p)
    tex_s=[]
    for word in tex_a:
        if (word not in stop_words):
            tex_s.append(word)
    tex_l = []
    for word in tex_s:
        tex_l.append(lemmetizer.lemmatize(word))
    tex_f = []
    for word in tex_l:
        if word not in unwanted_words:
            tex_f.append(word)
        else:
            pass
    data.iloc[i,1] = tex_f

#Named Entity Recognition
for i in range(0,len(data)):
    s= data.Body[i]
    doc = nlp(s)
    newString = s
    for e in reversed(doc.ents): #reversed to not modify the offsets of other entities when substituting
        start = e.start_char
        end = start + len(e.text)
        newString = newString[:start] + '[' + e.label_ + ']' + newString[end:]
    data.Body[i] = newString
    
data.head()

,Category,Search_column,Title,Body
0,Acceptance Emails,"[Elementary, School, Admission]",Elementary School Admission Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
1,Acceptance Emails,"[Academy, Admission]",Academy Admission Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
2,Acceptance Emails,"[College, Admissions, Recommendation]",College Admissions Recommendation Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
3,Acceptance Emails,"[Decline, Admission]",Decline Admission Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
4,Acceptance Emails,"[Harvard, Acceptance]",Harvard Acceptance Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...


# EDA

In [ ]:
data.shape

(1574, 4)

In [ ]:
data.columns

Index(['Category', 'Search_column', 'Title', 'Body'], dtype='object')

The dataset consists of 4 columns namely 'Category', 'Search_column', 'Title' and 'Body'.
<br>Dataset altogether consists of 1574 Email templates.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1574 entries, 0 to 1573
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Category       1574 non-null   object
 1   Search_column  1574 non-null   object
 2   Title          1574 non-null   object
 3   Body           1574 non-null   object
dtypes: object(4)
memory usage: 49.3+ KB


As we can see that there are no null values.

In [ ]:
print('Length: '+str(len(data.Category.unique()))+'\n')
data.Category.unique()

Length: 53



array(['Acceptance Emails', 'Admission Emails', 'Announcement Emails',
       'Application Emails', 'Appreciation Emails',
       'Authorization Emails', 'Banking Emails', 'Birthday Email',
       'Business Emails', 'Cancellation Email', 'Certification Email',
       'Claim Email', 'Complaint Emails', 'Condolence Emails',
       'Confirmation Email', 'Congratulation Emails', 'Consumer Emails',
       'Customer Emails', 'Decline Email', 'Emails', 'Employment Emails',
       'Finance Emails', 'Formal Email', 'General Emails',
       'Government Emails', 'Grievance Emails', 'Holiday Emails',
       'Inquiry Email', 'Internship Email', 'Interview Email',
       'Introduction Emails', 'Invitation Emails', 'Job Emails',
       'Leave Emails', 'Marketing Emails', 'Order Email',
       'Permission Email', 'Personal Emails', 'Recommendation Emails',
       'Reference Emails', 'Request Emails', 'Resignation Emails',
       'Retirement Emails', 'Sales & Marketing Emails', 'Sales Emails',
       '

As we can see that there are 53 Categories as shown above.

The title column shows the title of each Email

The Search_Column consists of the tokens from the title column for matching the keywords.

The Body column consists of templates of the email.

# Search Term Input and Processing

In [ ]:
g_text = input('Search Email Templates:\n\n')

Search Email Templates:

Sick Leave Email to boss


In [ ]:
text=''
text = re.sub(r'[^\w\s]','',g_text)
text

'Sick Leave Email to boss'

In [ ]:
#text=''
#for char in g_text:
#    if (char not in punctuations):
#        text = text+char
#text

In [ ]:
text = text.strip()
text

'Sick Leave Email to boss'

In [ ]:
text_a = nltk.word_tokenize(text)
text_a

['Sick', 'Leave', 'Email', 'to', 'boss']

In [ ]:
text_s=[]
for word in text_a:
    if (word not in stop_words):
        text_s.append(word)
        
text_s

['Sick', 'Leave', 'Email', 'boss']

In [ ]:
text_l = []
for word in text_s:
#    print(lemmetizer.lemmatize(word))
    if word == 'boss':
        text_l.append(word)
    else: 
        text_l.append(lemmetizer.lemmatize(word))
text_l

['Sick', 'Leave', 'Email', 'boss']

In [ ]:
text_f = []
for word in text_l:
    if word not in unwanted_words:
        text_f.append(word)
    else:
        pass
text_f

['Sick', 'Leave', 'boss']

## Searching for keywords in Search_column

In [ ]:
#Searching for keywords in the search column
result_in = []
for word in text_f:
    for i in range (0,len(data)):
        for j in range (0,len(data.iloc[i,1])):
            if data.iloc[i,1][j] not in text_f:
                pass
            else:
                result_in.append(int(i))
result_in.sort(reverse=False)
result_in

[129,
 129,
 129,
 257,
 257,
 257,
 978,
 978,
 978,
 979,
 979,
 979,
 980,
 980,
 980,
 981,
 981,
 981,
 982,
 982,
 982,
 983,
 983,
 983,
 984,
 984,
 984,
 985,
 985,
 985,
 986,
 986,
 986,
 987,
 987,
 987,
 989,
 989,
 989,
 990,
 990,
 990,
 991,
 991,
 991,
 995,
 995,
 995,
 997,
 997,
 997,
 998,
 998,
 998,
 999,
 999,
 999,
 1000,
 1000,
 1000,
 1001,
 1001,
 1001,
 1003,
 1003,
 1003,
 1004,
 1004,
 1004,
 1203,
 1203,
 1203,
 1486,
 1486,
 1486]

In [ ]:
x = data.iloc[result_in]
x['priority'] = np.NaN
x.head()

,Category,Search_column,Title,Body,priority
129,Application Emails,"[Marriage, Leave, Application]",Marriage Leave Application,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,NaN
129,Application Emails,"[Marriage, Leave, Application]",Marriage Leave Application,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,NaN
129,Application Emails,"[Marriage, Leave, Application]",Marriage Leave Application,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,NaN
257,Business Emails,"[Short, Request, Leave, Absence]",Short Request for Leave of Absence,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,NaN
257,Business Emails,"[Short, Request, Leave, Absence]",Short Request for Leave of Absence,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,NaN


In [ ]:
p = x.index.value_counts()
p

1004    3
985     3
257     3
1203    3
1486    3
978     3
979     3
980     3
981     3
982     3
983     3
984     3
986     3
1003    3
987     3
989     3
990     3
991     3
995     3
997     3
998     3
999     3
1000    3
1001    3
129     3
dtype: int64

In [ ]:
x = x.reset_index()
p = p.reset_index()
p = p.rename(columns={0:'priority'},inplace=False)

In [ ]:
x.head()

,index,Category,Search_column,Title,Body,priority
0,129,Application Emails,"[Marriage, Leave, Application]",Marriage Leave Application,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,NaN
1,129,Application Emails,"[Marriage, Leave, Application]",Marriage Leave Application,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,NaN
2,129,Application Emails,"[Marriage, Leave, Application]",Marriage Leave Application,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,NaN
3,257,Business Emails,"[Short, Request, Leave, Absence]",Short Request for Leave of Absence,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,NaN
4,257,Business Emails,"[Short, Request, Leave, Absence]",Short Request for Leave of Absence,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,NaN


In [ ]:
p.head()

,index,priority
0,1004,3
1,985,3
2,257,3
3,1203,3
4,1486,3


In [ ]:
for i in range(0,len(x)):
    ind = x['index'][i] 
    for j in range(0,len(p)):
        if p['index'][j] == ind:
            pr = p['priority'][j]
        else:
            pass
    x['priority'][i] = pr
x.head()

,index,Category,Search_column,Title,Body,priority
0,129,Application Emails,"[Marriage, Leave, Application]",Marriage Leave Application,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,3.0
1,129,Application Emails,"[Marriage, Leave, Application]",Marriage Leave Application,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,3.0
2,129,Application Emails,"[Marriage, Leave, Application]",Marriage Leave Application,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,3.0
3,257,Business Emails,"[Short, Request, Leave, Absence]",Short Request for Leave of Absence,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,3.0
4,257,Business Emails,"[Short, Request, Leave, Absence]",Short Request for Leave of Absence,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...,3.0


# Results


In [ ]:
x = x.sort_values(by=['priority'],ascending=False)
x = x.drop_duplicates(subset=['index'])
x = x.drop(['index','Search_column','priority'],axis=1)
x = x.reset_index(drop=True)
x.head()

,Category,Title,Body
0,Application Emails,Marriage Leave Application,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
1,Leave Emails,Marriage Leave Application,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
2,Leave Emails,Sample Leave Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
3,Leave Emails,Maternity Leave Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...
4,Leave Emails,Leave application Email,To: [Email id]\n\nSubject: [Subject]\n\n[Salut...


In [ ]:
print('Category: ',x.Category[0],'\n')
print('Title: ',x.Title[0])
print('____________________________________________________________________________________________________________________________\n')
print(x.Body[0])

Category:  Application Emails 

Title:  Marriage Leave Application
____________________________________________________________________________________________________________________________

To: [Email id]

Subject: [Subject]

[Salutation]

I am [PERSON] and I work as the Chief Engineer with your team. I take immense pleasure in inviting you to my marriage with [PERSON] on the [ORDINAL] of May. I will be sending you the personal invitation in due course of time. The actual reason is a request to you to grant me a leave of absence for [DATE] starting from the [ORDINAL] of May. Kindly treat my absence as informed leave and oblige.

[Closure]

[Name/Signature]
[Designation]
[Organization]



### Just for future reference

In [ ]:
df = pd.DataFrame(data.Category.value_counts())
df = df.reset_index()
df = df.rename(columns={'index':'Category','Category':'N_Letters'})
df

,Category,N_Letters
0,Business Emails,70
1,Personal Emails,70
2,Customer Emails,70
3,General Emails,64
4,Sales & Marketing Emails,60
5,Finance Emails,51
6,Thank You Emails,50
7,Complaint Emails,50
8,Congratulation Emails,50
9,Condolence Emails,49
